In [161]:
#!pip install pyTelegramBotAPI
!pip install python-telegram-bot


In [ ]:
import pandas as pd
import numpy as np
import telebot
from telebot import types
import datetime


# __________________ Data Load&Settings ________________

df = pd.read_csv('G:/Мой диск/MY/Образование/Data Sience/Проекты/проект чат-бот Ветмед/base.csv', sep = ';')
df['category'] = df['category'].str.strip()

#создание таблицы - корзины заказа. Вероятно ДАЛЕЕ требуется сделать отдельные таблицы на клиента, корзину, все заказы

basket = pd.DataFrame(columns = ['name', 'type', 'price','qual_order']) 
#basket = pd.DataFrame(columns = ['id_basket', 'name', 'type', 'price','qual_order']) #'status' пока исключен

#allowed_status = ['Оформлен', 'Подтвержден', 'Укомплектован', 'Отправлен', 'Получен'] #тут заданы варианты переменной статус
#basket['status'] = pd.Categorical(basket['status'], categories=allowed_status)


#таблица профиля клиента
client_profile = pd.DataFrame(columns = ['id_basket', 'id_client','name_client',
                                         'second_name_client', 'time_data','tel_client'])




# __________________ Start bot ________________


#код в отдельной переменной чтобы ДАЛЕЕ вынести в отдельный файл для безопасности
code = '********'

bot = telebot.TeleBot(code)


    
@bot.message_handler(commands = ['start'])
def start(message):
    main(message)
    


def main(message):

    global basket
    global count_orders 
    global basket_number
    global phone 
    global adress 
    
    basket = pd.DataFrame(columns=basket.columns)
    count_orders = 0 #счетчик заказов (id_basket) В БУДУЩЕМ нужно сделать отдельную таблицу и вести в ней учет всех заказов в независимости от работы бота
    basket_number = 0 #номер позиции в заказе
    phone = ""
    adress = ""
    first_step(message)

def first_step(message):
    markup = types.ReplyKeyboardMarkup()
    btn1 = types.KeyboardButton('По категориям')
    btn2 = types.KeyboardButton('По букве')
    markup.row(btn1, btn2)
    bot.send_message(
        message.chat.id,
        'Отлично. Вы можете искать лекарства по категориям или по первой букве названия. Выберите как вам удобнее',
        reply_markup=markup
    )
    bot.register_next_step_handler(message, handle_choice)

def handle_choice(message):
    if message.text == 'По категориям':
        select_category(message)
    elif message.text == 'По букве':
        select_letter(message)
    else:
        bot.send_message(
            message.chat.id,
            'Упс, ошибочка. Нажмите одну из кнопок'
        )
        first_step(message)

def select_category(message):
    markup = types.ReplyKeyboardMarkup()
    btn_c_1 = types.KeyboardButton('В рот')
    btn_c_2 = types.KeyboardButton('В ухо')
    btn_c_3 = types.KeyboardButton('В попу')
    markup.row(btn_c_1, btn_c_2)
    markup.row(btn_c_3)
    bot.send_message(
        message.chat.id,
        'Выберите интересующую вас категорию',
        reply_markup=markup
    )
    bot.register_next_step_handler(message, cont_category)

def select_letter(message):
    markup = types.ReplyKeyboardMarkup()
    btn_l_1 = types.KeyboardButton('A B C D')
    btn_l_2 = types.KeyboardButton('E F G H I')
    btn_l_3 = types.KeyboardButton('J K L M N')
    btn_l_4 = types.KeyboardButton('O P Q R')
    btn_l_5 = types.KeyboardButton('S T U V')
    btn_l_6 = types.KeyboardButton('W X Y Z')
    markup.row(btn_l_1, btn_l_2)
    markup.row(btn_l_3, btn_l_4)
    markup.row(btn_l_5, btn_l_6)
    bot.send_message(
        message.chat.id,
        'Выберите интересующую вас букву',
        reply_markup=markup
    )
    bot.register_next_step_handler(message, cont_letter)


#_______________функция для выбора препаратов по категории___________________________________

def cont_category (message):
    
    global df
    
    category_name = message.text  # Получаем текстовое сообщение от пользователя
    df_c = df.loc[(df['category'] == category_name) & (df['quality'] > 0), ['id', 'name', 'type', 'price']]
    if df_c.empty:
        bot.send_message(message.chat.id, 'Упс, ошибочка. Нажмите одну из кнопок')
        select_letter(message)
    else:
        bot.send_message(
            message.chat.id,
            f'В настоящее время в наличии следующие лекарства:\n') #{df_c.to_string(index=False)}')
         
        markup = types.InlineKeyboardMarkup()
        for index, row in df_c.iterrows():
            btn_text = f'{row["name"]} - {row["type"]} - {row["price"]}' #тут мы создаем список кнопок
            btn = types.InlineKeyboardButton(btn_text, callback_data=row["id"])
            markup.add(btn)
        # Добавление кнопки "Назад"
        btn_back = types.InlineKeyboardButton('Назад',callback_data='back')
        markup.add(btn_back)
        bot.send_message(message.chat.id, 'Выберите товар:', reply_markup=markup)

        

    
#функция для выбора препаратов по первой букве
def cont_letter (message):
    global df
    letters = message.text  # Получаем текстовое сообщение от пользователя
    df_l = df.loc[(df['name'].str.contains(f'^[{letters}]', regex=True)) & (df['quality'] > 0), ['id', 'name', 'type', 'price']]
    if df_l.empty:
        bot.send_message(message.chat.id, 'Упс, ошибочка. Нажмите одну из кнопок')
        select_letter(message)
    else:
        bot.send_message(
        message.chat.id,
        f'В настоящее время в наличии следующие лекарства:\n')  #{df_l.to_string(index=False)}') 
        markup = types.InlineKeyboardMarkup() #ReplyKeyboardMarkup(resize_keyboard=True)
        for index, row in df_l.iterrows():
            btn_text = f'{row["name"]} - {row["type"]} - {row["price"]}' #тут мы создаем список кнопок
            btn = types.InlineKeyboardButton(btn_text, callback_data=row['id'])
            markup.add(btn)
        # Добавление кнопки "Назад"
        btn_back = types.InlineKeyboardButton('Назад',callback_data='back')
        markup.add(btn_back)

        bot.send_message(message.chat.id, 'Выберите товар:', reply_markup=markup)
    
     
    
# __________________ Basket  добавляем товар в корзину, спрашиваем количество  ________________
    
@bot.callback_query_handler(func=lambda call: True)    
def process_basket(call):
    global basket
    global basket_number
    global count_orders
    global df

    id_number = int(call.data)
    selected_vitamin = df[df['id'] == id_number]
    
    if selected_vitamin.empty:
        bot.send_message(message.chat.id, 'Упс, ошибочка. Выберите лекарство')
        handle_choice(message)
    
    else:
        basket = basket.append({
                'name': selected_vitamin['name'].values[0] ,
                'type': selected_vitamin['type'].values[0] ,
                'price': selected_vitamin['price'].values[0] ,
                
                }, ignore_index=True)

        choice = basket[['name','type','price']].to_string(header=False, index=False) 
        bot.send_message(
        call.message.chat.id,
        f'Вы выбрали:\n{choice}\nСколько упаковок вы хотите приобрести?\nЕсли не хотите, наберите 0'
        )
        
        bot.register_next_step_handler(call.message, basket_add)
 

      
    

#________________________функция добавление количества в корзину в корзину ________________________

def basket_add(message):
    global basket
    global basket_number
    global count_orders
    global df
        
    try:
        user_input = int(message.text)  # Попытка преобразовать текст в число
        if user_input > 0:
            basket.loc[basket_number, ['qual_order']] = user_input

            choice = basket[['name','type','price','qual_order']].to_string(header=False, index=False) 

            basket_number += 1

            summ = (basket['price'] * basket['qual_order']).sum()

            bot.send_message(
                message.chat.id,
                f'В вашей корзине:\n{choice}\nСумма вашего заказа:\n{summ}')

            markup = types.ReplyKeyboardMarkup()
            btn1 = types.KeyboardButton ('Продолжим покупки')
            btn2 = types.KeyboardButton ('Оформим заказ')
            markup.row(btn1, btn2)
            bot.send_message(message.chat.id, 
            'Продолжим покупки или оформим заказ?', 
            reply_markup=markup)
            bot.register_next_step_handler(message, buy_or_checkout)

        else:
            return first_step(message)  # Вызов функции main, если число не больше нуля
    
    except ValueError:
        return first_step(message)

    
#___________    ВОЗВРАТ В НАЧАЛО ДЛЯ ПРОДОЛЖЕНИЯ ПОКУПОК ИЛИ ОФОРМЛЕНИЕ ЗАКАЗА ______________
def buy_or_checkout(message):
    
    choice = message.text
    
    if choice == 'Продолжим покупки': 
        return first_step(message)

    elif choice == 'Оформим заказ': 
        bot.send_message(
        message.chat.id, 
        'Наш менеджер позвонит вам и подтвердит заказ\nВведите номер вашего телефона:')        
        bot.register_next_step_handler(message, phonede)

        
#___________    ОФОРМЛЕНИЕ ЗАКАЗА ______________

def phonede (message):
    global basket
    global phone
    global adress 
    
    phone = message.text
        
    bot.send_message(
    message.chat.id, 
    f'Введите адрес доставки:')   
    bot.register_next_step_handler(message, send_order)
    
    
def send_order(message):    
    global basket
    global phone
    global adress 
    
    
    adress = message.text
#    adress = adress.replace(" ", "-")
    
    summ = (basket['price'] * basket['qual_order']).sum()
    
    choice = basket[['name','type','price','qual_order']].to_string(header=False, index=False) 

    now = datetime.datetime.now()
    formatted_date_time = now.strftime('%Y-%m-%d %H:%M:%S')
    
    # Формируем текст сообщения о заказе
    message_text = (
        f'НОВЫЙ ЗАКАЗ\n'
        f'Состав:\n {choice}\n'
        f'Сумма: {summ}\n'
        f'Имя заказчика: {message.from_user.first_name}\n'
        f'Фамилия: {message.from_user.last_name}\n'
        f'Имя пользователя ТГ: {message.from_user.username}\n'
        f'Номер телефона: {phone}\n'
        f'Адрес: {adress}\n'
        f'Робот? {message.from_user.is_bot}\n'
        f'Дата и время запроса: {formatted_date_time}'
    )

    # Отправляем сообщение с информацией о заказе
    bot.send_message(-1002049337991, message_text)
    
 
    markup = types.ReplyKeyboardMarkup()
    btn1 = types.KeyboardButton ('Еще разик!')
    markup.row(btn1)
    
    message_text_adress = (
    f'{message.from_user.first_name}, мы приняли ваш заказ и передали его менеджеру\n'
    f'Состав:\n {choice}\n'
    f'Сумма: {summ}\n'
    f'Номер телефона: {phone}\n'
    f'Адрес: {adress}\n'
    f'Дата и время запроса: {formatted_date_time}'
    )
    
    bot.send_message( message.chat.id, message_text_adress, reply_markup=markup)
    bot.register_next_step_handler(message, phenix)
 
  
def phenix(message):
    main(message)

# Запуск бота
bot.infinity_polling()

2023-12-27 11:05:21,593 (__init__.py:960 MainThread) ERROR - TeleBot: "Infinity polling exception: HTTPSConnectionPool(host='api.telegram.org', port=443): Read timed out. (read timeout=25)"
2023-12-27 11:05:21,595 (__init__.py:962 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "C:\Users\OLEG\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\OLEG\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\OLEG\anaconda3\lib\http\client.py", line 1371, in getresponse
    response.begin()
  File "C:\Users\OLEG\anaconda3\lib\http\client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "C:\Users\OLEG\anaconda3\lib\http\client.py", line 280, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")


In [ ]:

@bot.message_handler(commands=['help'])
def main2(message):
    bot.send_message(message.chat.id, '<b>Да, жизнь сложная штука!</b>', parse_mode='html')